In [ ]:
# If you run on colab uncomment the following line
#!pip install git+https://github.com/clementchadebec/benchmark_VAE.git

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.distributions import MultivariateNormal

%load_ext autoreload
%autoreload 2

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from sklearn import datasets
from pythae.data.datasets import BaseDataset

train_moons = torch.from_numpy(datasets.make_moons(n_samples=10000, noise=0.05)[0].astype(np.float32))
eval_moons = torch.from_numpy(datasets.make_moons(n_samples=1000, noise=0.05)[0].astype(np.float32))

In [ ]:
from pythae.models.normalizing_flows import MAFConfig, MAF, NFModel, IAF, IAFConfig

prior = MultivariateNormal(torch.zeros(2).to(device), torch.eye(2).to(device))

conf = MAFConfig(input_dim=(2,), n_hidden_in_made=3, hidden_size=24, n_made_blocks=4, include_batch_norm=False)
flow = MAF(conf)

# conf = IAFConfig(input_dim=(2,), n_hidden_in_made=3, hidden_size=24, n_made_blocks=2, include_batch_norm=False)
# flow = IAF(conf)


contained_flow = NFModel(prior, flow)

In [ ]:
from pythae.trainers import BaseTrainer, BaseTrainerConfig
from pythae.pipelines import TrainingPipeline


train_config = BaseTrainerConfig(num_epochs=50, learning_rate=1e-3)
pipeline = TrainingPipeline(model=contained_flow, training_config=train_config)

In [ ]:
pipeline(
    train_data=train_moons,
    eval_data=eval_moons
)

In [ ]:
trained_flow = pipeline.trainer._best_model.flow
trained_flow.eval()

z_sample = prior.sample((128*8,))

z = trained_flow.inverse(z_sample).out.detach().cpu()
plt.scatter(train_moons[:,0], train_moons[:,1], c='b', s=20, alpha=0.5, label='true distribution')
plt.scatter(z[:,0], z[:,1], c='r', s=20, alpha=0.5, label='transformed gaussian')
plt.legend()

In [ ]:
#trained_flow = pipeline.trainer._best_model.flow
#trained_flow.eval()

maf_rec = MAF.load_from_folder(os.path.join(pipeline.trainer.training_dir, 'final_model')).to(device)

z = maf_rec.inverse(z_sample).out.detach().cpu()
plt.scatter(train_moons[:,0], train_moons[:,1], c='b', s=20, alpha=0.5, label='true distribution')
plt.scatter(z[:,0], z[:,1], c='r', s=20, alpha=0.5, label='transformed gaussian')
plt.legend()

In [ ]:
pipeline.trainer.training_dir, "dummy_output_dir/MAF_training_2022-04-04_16-02-43/final_model"

In [ ]:
os.path.join(pipeline.trainer.training_dir, 'trained_model'),"dummy_output_dir/MAF_training_2022-04-04_16-02-43/final_model"